In [6]:
import torch,tiktoken
from torch.utils.data import DataLoader, Dataset

In [4]:
##Create input target tensors
class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer, max_length,stride):
        self.input_ids = []
        self.target_ids = []

        #Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={'|eos|'})

        #Use sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0,len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    ##Dataloader requires Map-style or iterable style, dataloader will check the __getitem__ 1st and after that
    # after that only it will go and create input and target chunk.
    # getitem will clearly what input and target should be - its based on index

        
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self,idx): 
        return self.input_ids[idx],self.target_ids[idx]

In [5]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    ##Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    #Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    #Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [7]:
from pypdf import PdfReader
reader = PdfReader("war-and-peace.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text()

raw_text = ""
for page in reader.pages:
    raw_text += page.extract_text()

In [3]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [8]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride = max_length, shuffle=True
)
data_iter = iter(dataloader)
inputs,target = next(data_iter)

In [14]:
emb_input = token_embedding_layer(inputs)
print(inputs.shape, emb_input.shape)

torch.Size([8, 4]) torch.Size([8, 4, 256])


In [15]:
context_length = max_length
pos_emb_layer = torch.nn.Embedding(context_length,output_dim)
pos_emb = pos_emb_layer(torch.arange(max_length))
print(pos_emb.shape)

torch.Size([4, 256])


In [18]:
input_emb = emb_input+pos_emb
print(input_emb.shape)

torch.Size([8, 4, 256])
